# Cài đặt các thư viện cần thiết

In [ ]:
!pip install pandas numpy matplotlib seaborn
!pip install nltk spacy transformers
!pip install gensim bertopic wordcloud
!python -m spacy download en_core_web_sm

: 


# Import các thư viện cần thiết cho xử lý dữ liệu và phân tích

In [ ]:
import pandas as pd
import numpy as np
import nltk
import spacy
import re
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
nltk.download('stopwords')

# Lấy dữ liệu từ file csv và xem qua 

In [ ]:
url ='https://raw.githubusercontent.com/dD2405/Twitter_Sentiment_Analysis/master/train.csv'
df = pd.read_csv(url)
df = df.iloc[10000:20000]  
df.head()

# Tiền xử lý các đoạn text

In [ ]:
def clean_text(text):
    text = text.lower()
    text = re.sub(r'http\S+|www\S+|https\S+', '', text)
    text = re.sub(r'\W', ' ', text)
    text = re.sub(r'\s+', ' ', text)
    return text.strip()

df['clean_text'] = df['tweet'].apply(clean_text)
df.head()

# Phân tích cảm xúc giúp xác định thái độ (tích cực, tiêu cực, trung lập) trong mỗi bài đăng

In [ ]:
from nltk.sentiment import SentimentIntensityAnalyzer
nltk.download('vader_lexicon')

sia = SentimentIntensityAnalyzer()

def sentiment_score(text):
    score = sia.polarity_scores(text)
    if score['compound'] > 0:
        return 'Positive'
    elif score['compound'] < 0:
        return 'Negative'
    else:
        return 'Neutral'

df['sentiment'] = df['clean_text'].apply(sentiment_score)

# Vẽ biểu đồ số lượng các bài đăng theo các loại cảm xúc (Positive, Neutral, Negative)

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

sns.countplot(x='sentiment', data=df, palette='viridis')                      
plt.title('Sentiment Analysis')
plt.show()

Chia tập dữ liệu train và tập dữ liệu test

In [ ]:
X = df['clean_text']
y = df['sentiment']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Thêm các thư viện cần thiết 

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB

# Khởi tạo CountVectorizer với việc loại bỏ các stopwords (từ không mang nhiều ý nghĩa)
# Chuyển văn bản huấn luyện và kiểm tra thành dạng ma trận đặc trưng

In [ ]:
vectorizer = CountVectorizer(stop_words='english')
X_train_vec = vectorizer.fit_transform(X_train)
X_test_vec = vectorizer.transform(X_test)

# Khởi tạo mô hình Naive Bayes
# Huấn luyện mô hình với dữ liệu huấn luyện

In [ ]:
model = MultinomialNB()
model.fit(X_train_vec, y_train)

# Đánh giá mô hình

In [ ]:
y_pred = model.predict(X_test_vec)

print("Classification Report:")
print(classification_report(y_test, y_pred))

print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))

print("Accuracy Score:")
print(accuracy_score(y_test, y_pred))

# Sử dụng BERTopic để phân tích topic từ văn bản

In [ ]:
from bertopic import BERTopic

# Khởi tạo mô hình BERTopic
# Áp dụng mô hình topic cho tập huấn luyện

In [ ]:
topic_model = BERTopic()
topics, probs = topic_model.fit_transform(X_train.tolist())

# In ra thông tin về các topic

In [ ]:
print(topic_info)

In [ ]:
df['predicted_sentiment'] = model.predict(vectorizer.transform(df['clean_text']))
print(df.head())